This notebook researches how BitQuery GraphQL API interacts with Uniswap data.

# Imports

In [1]:
import os
from typing import Any, Dict, List

import pandas as pd
import requests

# Functions

In [3]:
def run_query(query: str):
    headers = {"X-API-KEY": os.environ["API_KEY"]}
    request = requests.post(
        "https://graphql.bitquery.io/", json={"query": query}, headers=headers
    )
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception(
            "Query failed and return code is {}.      {}".format(
                request.status_code, query
            )
        )

In [52]:
def json_to_df(data: List[Dict[Any, Any]]) -> pd.DataFrame:
    """
    Transform the data to Dataframe and set the time index.
    """
    df = pd.json_normalize(data, sep="_")
    df = df.set_index("timeInterval_minute")
    return df

# Get data

**1 min data example for WETH/USDT pair**

In [54]:
query = """
query{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: 20000, asc: "timeInterval.minute"}
      date: {since: "2023-01-01", till: "2023-01-10"}
      baseCurrency: {is: "0xdac17f958d2ee523a2206206994597c13d831ec7"}
      quoteCurrency: {is: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}
      exchangeName: {is: "Uniswap"}
     )
     {
      timeInterval {
      minute(count: 1)
      }
    baseCurrency {
      symbol
      address
    }
    baseAmount(in: USD)
    quoteCurrency {
      symbol
      address
    }
    quoteAmount(in: USD)
    trades: count
    quotePrice
    maximum_price: quotePrice(calculate: maximum)
    minimum_price: quotePrice(calculate: minimum)
    open_price: minimum(of: block, get: quote_price)
    close_price: maximum(of: block, get: quote_price)
    }
  }
}
"""
# Get the data
result = run_query(query)

In [55]:
df = json_to_df(result["data"]["ethereum"]["dexTrades"])
df

,baseAmount,quoteAmount,trades,quotePrice,maximum_price,minimum_price,open_price,close_price,baseCurrency_symbol,baseCurrency_address,quoteCurrency_symbol,quoteCurrency_address
timeInterval_minute,,,,,,,,,,,,
2023-01-01 00:01:00,10509.147066,10503.845910,4,0.000836,0.000836,0.000834,0.0008356187916129935,0.0008355794241285353,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2023-01-01 00:03:00,1552.106180,1548.453089,2,0.000834,0.000834,0.000834,0.0008341360983627016,0.0008340226311907534,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2023-01-01 00:04:00,2654.976338,2656.218968,2,0.000836,0.000836,0.000836,0.0008364240918835278,0.0008364342579494035,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2023-01-01 00:05:00,3088.328830,3087.887825,2,0.000836,0.000836,0.000836,0.0008355922010752534,0.0008364257533853419,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2023-01-01 00:06:00,15304.270589,15312.594638,1,0.000836,0.000836,0.000836,0.0008364889265664506,0.0008364889265664506,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-10 10:25:00,2956.316732,2955.046225,6,0.000752,0.000752,0.000751,0.0007511759225325143,0.0007511741510742848,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2023-01-10 10:26:00,3112.923878,3110.173661,3,0.000751,0.000752,0.000751,0.0007519258343342432,0.0007509539252893075,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2023-01-10 10:27:00,74389.971388,74287.649904,5,0.000751,0.000751,0.000751,0.0007509491077693408,0.000751334782876887,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


The query was made at 2023-01-10 10:33:00, so real-time delay is ~3-4 minutes.

**Get historical data Uniswap from november 2018 with no token pairs filter**

In [58]:
query = """
query{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: 20000, asc: "timeInterval.minute"}
      date: {since: "2018-11-01", till: "2018-12-01"}
      exchangeName: {is: "Uniswap"}
     )
     {
      timeInterval {
      minute(count: 1)
      }
    baseCurrency {
      symbol
      address
    }
    baseAmount(in: USD)
    quoteCurrency {
      symbol
      address
    }
    quoteAmount(in: USD)
    trades: count
    quotePrice
    maximum_price: quotePrice(calculate: maximum)
    minimum_price: quotePrice(calculate: minimum)
    open_price: minimum(of: block, get: quote_price)
    close_price: maximum(of: block, get: quote_price)
    }
  }
}
"""
# Get the data
result = run_query(query)

In [59]:
df = json_to_df(result["data"]["ethereum"]["dexTrades"])
df

,baseAmount,quoteAmount,trades,quotePrice,maximum_price,minimum_price,open_price,close_price,baseCurrency_symbol,baseCurrency_address,quoteCurrency_symbol,quoteCurrency_address
timeInterval_minute,,,,,,,,,,,,
2018-11-02 07:48:00,0.001982,0.000000,1,4.984503e+03,4.984503e+03,4.984503e+03,4984.503045046409,4984.503045046409,ETH,-,HAY,0xfa3e941d1f6b7b10ed84a0c211bfa8aee907965e
2018-11-02 07:48:00,0.000000,0.001982,1,2.006218e-04,2.006218e-04,2.006218e-04,0.00020062180541624875,0.00020062180541624875,HAY,0xfa3e941d1f6b7b10ed84a0c211bfa8aee907965e,ETH,-
2018-11-02 08:11:00,0.000000,0.003956,1,5.034985e-08,5.034985e-08,5.034985e-08,5.034985141189456e-08,5.034985141189456e-08,JCD,0x0ed024d39d55e486573ee32e583bc37eb5a6271f,ETH,-
2018-11-02 08:11:00,0.003956,0.000000,1,1.986103e+07,1.986103e+07,1.986103e+07,19861031.799664095,19861031.799664095,ETH,-,JCD,0x0ed024d39d55e486573ee32e583bc37eb5a6271f
2018-11-02 08:13:00,0.000000,0.019755,1,5.155041e-08,5.155041e-08,5.155041e-08,5.1550414235835117e-08,5.1550414235835117e-08,JCD,0x0ed024d39d55e486573ee32e583bc37eb5a6271f,ETH,-
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-01 21:17:00,23.492048,23.768617,1,3.108533e-01,3.108533e-01,3.108533e-01,0.31085329154884456,0.31085329154884456,ETH,-,MKR,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
2018-12-01 21:19:00,23.774299,23.554223,1,3.224695e+00,3.224695e+00,3.224695e+00,3.224694840532918,3.224694840532918,MKR,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ETH,-
2018-12-01 21:19:00,23.554223,23.774299,1,3.101069e-01,3.101069e-01,3.101069e-01,0.31010686264959525,0.31010686264959525,ETH,-,MKR,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2


**Get bigger: download all swap data for today (2023-01-10)**

In [62]:
query = """
query{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: 100000, asc: "timeInterval.minute"}
      date: {since: "2023-01-10"}
      exchangeName: {is: "Uniswap"}
     )
     {
      timeInterval {
      minute(count: 1)
      }
    baseCurrency {
      symbol
      address
    }
    baseAmount(in: USD)
    quoteCurrency {
      symbol
      address
    }
    quoteAmount(in: USD)
    trades: count
    quotePrice
    maximum_price: quotePrice(calculate: maximum)
    minimum_price: quotePrice(calculate: minimum)
    open_price: minimum(of: block, get: quote_price)
    close_price: maximum(of: block, get: quote_price)
    }
  }
}
"""
# Get the data
result = run_query(query)

In [63]:
result

{'data': {'ethereum': {'dexTrades': None}},
 'errors': [{'message': 'ActiveRecord::ActiveRecordError: Response code: 500:\nCode: 396, e.displayText() = DB::Exception: Limit for result exceeded, max rows: 25.00 thousand, current rows: 61.70 thousand (version 20.8.11.17 (official build))\n',
   'locations': [{'line': 4, 'column': 5}],
   'path': ['ethereum', 'dexTrades'],
   'error_type': 'server',
   'query_id': '8FLRXsBiDyBA7D7m'}]}

It seems like we have a limitation for number of rows by one query, let's try to write the loop and export every 25k rows.

In [67]:
# Use limit and offset parameters.
query = """query{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: 25000, offset: %d, asc: "timeInterval.minute"}
      date: {since: "2023-01-10"}
      exchangeName: {is: "Uniswap"}
     )
     {
      timeInterval {
      minute(count: 1)
      }
    baseCurrency {
      symbol
      address
    }
    baseAmount(in: USD)
    quoteCurrency {
      symbol
      address
    }
    quoteAmount(in: USD)
    trades: count
    quotePrice
    maximum_price: quotePrice(calculate: maximum)
    minimum_price: quotePrice(calculate: minimum)
    open_price: minimum(of: block, get: quote_price)
    close_price: maximum(of: block, get: quote_price)
    }
  }
}
"""

limit = 25000
offset = 0
data_dfs = []

for i in range(3):
    # Error message above says there are 61k rows, so 3 iterations by 25k would be enough.
    formatted_query = query % offset
    result = run_query(formatted_query)
    df = json_to_df(result["data"]["ethereum"]["dexTrades"])
    data_dfs.append(df)
    offset += 25000

full_data = pd.concat(data_dfs)

In [68]:
full_data

,baseAmount,quoteAmount,trades,quotePrice,maximum_price,minimum_price,open_price,close_price,baseCurrency_symbol,baseCurrency_address,quoteCurrency_symbol,quoteCurrency_address
timeInterval_minute,,,,,,,,,,,,
2023-01-10 00:00:00,1224.931872,1219.843144,1,1.478314e+02,1.478314e+02,1.478314e+02,147.83144645561345,147.83144645561345,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wNXM,0x0d438f3b5175bebc262bf23753c1e53d03432bde
2023-01-10 00:00:00,0.000000,47.360859,1,2.866490e-05,2.866490e-05,2.866490e-05,2.8664898957457876e-05,2.8664898957457876e-05,DD,0x246e571bef4681e1a37da199763d72d8bbbeb31c,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2023-01-10 00:00:00,72.585248,73.424473,1,1.367934e+06,1.367934e+06,1.367934e+06,1367934.077913995,1367934.077913995,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,imgnAI,0xa735a3af76cc30791c61c10d585833829d36cbe0
2023-01-10 00:00:00,91.715003,0.000000,1,3.845421e+04,3.845421e+04,3.845421e+04,38454.212993146415,38454.212993146415,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,HDWY,0xab22e79f2dfc2e572223780fa425c827b0892170
2023-01-10 00:00:00,13.217799,0.000000,1,4.711646e+08,4.711646e+08,4.711646e+08,471164564.3726406,471164564.3726406,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,SHIBABOSU,0xd3f81c4435b796d6fbb8df90db636cc3c68d94b8
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-10 10:37:00,626.978963,626.881253,1,9.998886e-01,9.998886e-01,9.998886e-01,0.9998886240595996,0.9998886240595996,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,DAI,0x6b175474e89094c44da98b954eedeac495271d0f
2023-01-10 10:37:00,680.169003,0.000000,2,4.806748e+04,5.348123e+04,4.732852e+04,53481.225506229755,47328.52408237778,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,KENZO,0xf25ae30a78733b04c23d624b5f71dfdaabeaac6d
2023-01-10 10:37:00,198.447744,198.905225,5,1.333232e+03,1.334110e+03,1.326118e+03,1333.1596996099222,1326.1178306189263,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48


The query was made at 2023-01-10 10:42:00, so real-time delay is 5 min.